In [2]:
!pip install -U tensorflow-datasets

Requirement already up-to-date: tensorflow-datasets in /usr/local/lib/python3.6/dist-packages (2.0.0)


In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import math
%matplotlib inline

In [0]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [5]:
dataset,metadata = tfds.load('fashion_mnist',as_supervised=True,with_info=True)
train_dataset,test_dataset = dataset['train'],dataset['test']

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.0.incompleteYG6LYY/fashion_mnist-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.0.incompleteYG6LYY/fashion_mnist-test.tfrecord


Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.0. Subsequent calls will reuse this data.


In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

In [7]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

Number of training examples: 60000
Number of test examples:     10000


In [0]:
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

# The map function applies the normalize function to each element in the train
# and test datasets
train_dataset =  train_dataset.map(normalize)
test_dataset  =  test_dataset.map(normalize)

# The first time you use the dataset, the images will be loaded from disk
# Caching will keep them in memory, making training faster
train_dataset =  train_dataset.cache()
test_dataset  =  test_dataset.cache()

In [12]:
#model building
l0 = tf.keras.layers.Flatten(input_shape=(28,28,1))
l1 = tf.keras.layers.Dense(units=128,activation=tf.nn.relu)
l2 = tf.keras.layers.Dense(10,activation=tf.nn.softmax)
model = tf.keras.Sequential([l0,l1,l2])
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [0]:
BATCH_SIZE=32
train_dataset=train_dataset.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [14]:
model.fit(train_dataset,epochs=20,steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Train on 1875 steps
Epoch 1/20
1875/1875 [==============================] - 17s 9ms/step - loss: 1.6754 - acc: 0.7945
Epoch 2/20
1875/1875 [==============================] - 4s 2ms/step - loss: 1.6197 - acc: 0.8436
Epoch 3/20
1875/1875 [==============================] - 4s 2ms/step - loss: 1.6069 - acc: 0.8567
Epoch 4/20
1875/1875 [==============================] - 4s 2ms/step - loss: 1.5995 - acc: 0.8630
Epoch 5/20
1875/1875 [==============================] - 4s 2ms/step - loss: 1.5956 - acc: 0.8666
Epoch 6/20
1875/1875 [==============================] - 4s 2ms/step - loss: 1.5906 - acc: 0.8710
Epoch 7/20
1875/1875 [==============================] - 4s 2ms/step - loss: 1.5859 - acc: 0.8763
Epoch 8/20
1875/1875 [==============================] - 4s 2ms/step - loss: 1.5827 - acc: 0.8787
Epoch 9/20
1875/1875 [==============================] - 4s 2ms/step - loss: 1.5813 - acc: 0.8810
Epoch 10/20
1875/1875 [==============================] - 4s 2ms/step - loss: 1.5737 - acc: 0.8876
Epoch 11

In [17]:
test_loss,test_accuracy = model.evaluate(test_dataset,steps=math.ceil(num_train_examples/BATCH_SIZE))
print("Accuracy on Test:" ,test_accuracy)

 312/1875 [===>..........................] - ETA: 14s - loss: 1.5837 - acc: 0.8774Accuracy on Test: 0.8772
